In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[6],4
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,6.5e-24
molecule,{'co2': 0.0004},co2
ng_adju,-,"[0, 0]"
ng_refs,-,"[1, 2]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.000758                    1 -0.000777
0.000750        2 -0.000760                    2 -0.000799
0.001052        3 -0.000764                    3 -0.000779
0.001476        4 -0.000768                    4 -0.000782
0.002070        5 -0.000772                    5 -0.000788
0.002904        6 -0.000775                    6 -0.000792
0.004074        7 -0.000777                    7 -0.000798
0.005714        8 -0.000779                    8 -0.000801
0.008015        9 -0.000781                    9 -0.000805
0.011243       10 -0.000781                   10 -0.000809
0.015771       11 -0.000781                   11 -0.000817
0.022122       12 -0.000781                   12 -0.000823
0.031031       13 -0.000780                   13 -0.000827
0.043528       14 -0.000777                   14 -0.000832
0.061057       15 -0.000765                   15 -0.000839
0.085645       16 -0.000680                   16 -0.000822
0.120136       17 -0.000381                   17 -0.000682
0.168516       18  0.000260                   18 -0.000318
0.236378       19  0.001425                   19  0.000387
0.331549       20  0.003374                   20  0.001596
0.465100       21  0.006455                   21  0.003555
0.652400       22  0.011099                   22  0.006795
0.915100       23  0.016166                   23  0.010424
1.283650       24  0.016278                   24  0.010641
1.800600       25  0.011654                   25  0.007670
2.525700       26  0.007232                   26  0.004757
3.542800       27  0.004017                   27  0.002574
4.969550       28  0.001869                   28  0.001079
6.970850       29  0.000534                   29  0.000117
9.778100       30 -0.000275                   30 -0.000491
13.715850      31 -0.000768                   31 -0.000878
19.239350      32 -0.001044                   32 -0.001101
26.987250      33 -0.001168                   33 -0.001208
37.855300      34 -0.001224                   34 -0.001263
53.100050      35 -0.001257                   35 -0.001294
73.887500      36 -0.001279                   36 -0.001313
97.662500      37 -0.001307                   37 -0.001337
121.437500     38 -0.001347                   38 -0.001355
145.212500     39 -0.001385                   39 -0.001339
168.987500     40 -0.001436                   40 -0.001361
192.762500     41 -0.001530                   41 -0.001416
216.537500     42 -0.001629                   42 -0.001547
240.312500     43 -0.001674                   43 -0.001621
264.087500     44 -0.001654                   44 -0.001620
287.862500     45 -0.001563                   45 -0.001545
311.637500     46 -0.001398                   46 -0.001402
335.412500     47 -0.001159                   47 -0.001169
359.187500     48 -0.000842                   48 -0.000872
382.962500     49 -0.000448                   49 -0.000505
406.737500     50  0.000021                   50 -0.000061
430.512500     51  0.000561                   51  0.000452
454.287500     52  0.001166                   52  0.001029
478.062500     53  0.001832                   53  0.001675
501.837500     54  0.002554                   54  0.002387
525.612500     55  0.003334                   55  0.003130
549.387500     56  0.004172                   56  0.003938
573.162500     57  0.005070                   57  0.004816
596.937500     58  0.006028                   58  0.005766
620.712500     59  0.007045                   59  0.006790
644.487500     60  0.008123                   60  0.007855
668.262500     61  0.009258                   61  0.008962
692.037500     62  0.010452                   62  0.010134
715.812500     63  0.011704                   63  0.011375
739.587500     64  0.013011                   64  0.012681
763.362500     65  0.014370                   65  0.014046
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -61.259575  0.000000e+00 -61.259575 -61.296249   
0.000624    2     -61.259575  3.165274e-08 -61.259575 -61.296249   
0.000876    3     -61.259575  4.462509e-08 -61.259575 -61.296249   
0.001229    4     -61.259575  6.350668e-08 -61.259575 -61.296249   
0.001723    5     -61.259575  9.105809e-08 -61.259575 -61.296249   
0.002417    6     -61.259575  1.312444e-07 -61.259575 -61.296249   
0.003391    7     -61.259575  1.898588e-07 -61.259575 -61.296250   
0.004757    8     -61.259575  2.753433e-07 -61.259575 -61.296250   
0.006672    9     -61.259576  3.999923e-07 -61.259575 -61.296250   
0.009359    10    -61.259576  5.817322e-07 -61.259576 -61.296250   
0.013128    11    -61.259577  8.466751e-07 -61.259576 -61.296251   
0.018415    12    -61.259577  1.232869e-06 -61.259576 -61.296252   
0.025830    13    -61.259579  1.795735e-06 -61.259577 -61.296253   
0.036232    14    -61.259580  2.616023e-06 -61.259578 -61.296255   
0.050823    15    -61.259583  3.814291e-06 -61.259579 -61.296257   
0.071291    16    -61.259587  5.706647e-06 -61.259581 -61.296260   
0.100000    17    -61.259593  9.330929e-06 -61.259583 -61.296266   
0.140271    18    -61.259603  1.712085e-05 -61.259586 -61.296275   
0.196760    19    -61.259617  3.375541e-05 -61.259584 -61.296289   
0.275997    20    -61.259639  6.879693e-05 -61.259571 -61.296311   
0.387100    21    -61.259667  1.414804e-04 -61.259526 -61.296342   
0.543100    22    -61.259696  2.900180e-04 -61.259407 -61.296381   
0.761700    23    -61.259706  5.873638e-04 -61.259119 -61.296423   
1.068500    24    -61.259661  1.129596e-03 -61.258532 -61.296452   
1.498800    25    -61.259583  1.881589e-03 -61.257702 -61.296489   
2.102400    26    -61.259560  2.691593e-03 -61.256869 -61.296593   
2.949000    27    -61.259640  3.497292e-03 -61.256143 -61.296801   
4.136600    28    -61.259859  4.280543e-03 -61.255578 -61.297143   
5.802500    29    -61.260249  5.039782e-03 -61.255210 -61.297647   
8.139200    30    -61.260851  5.790202e-03 -61.255062 -61.298353   
11.417000   31    -61.261720  6.551462e-03 -61.255168 -61.299310   
16.014700   32    -61.262920  7.332933e-03 -61.255587 -61.300581   
22.464000   33    -61.264532  8.147146e-03 -61.256384 -61.302242   
31.510500   34    -61.266689  9.053029e-03 -61.257636 -61.304428   
44.200100   35    -61.269615  1.013920e-02 -61.259475 -61.307367   
62.000000   36    -61.273588  1.146158e-02 -61.262126 -61.311346   
85.775000   37    -61.278742  1.301461e-02 -61.265727 -61.316513   
109.550000  38    -61.283905  1.449634e-02 -61.269409 -61.321685   
133.325000  39    -61.289208  1.600444e-02 -61.273203 -61.326920   
157.100000  40    -61.294641  1.753582e-02 -61.277105 -61.332090   
180.875000  41    -61.300310  1.916111e-02 -61.281149 -61.337395   
204.650000  42    -61.306621  2.116450e-02 -61.285456 -61.343143   
228.425000  43    -61.313950  2.390651e-02 -61.290044 -61.349942   
252.200000  44    -61.322353  2.759304e-02 -61.294759 -61.357813   
275.975000  45    -61.331793  3.237569e-02 -61.299417 -61.366672   
299.750000  46    -61.342220  3.840145e-02 -61.303818 -61.376439   
323.525000  47    -61.353572  4.581506e-02 -61.307757 -61.387080   
347.300000  48    -61.365778  5.475828e-02 -61.311019 -61.398379   
371.075000  49    -61.378759  6.536855e-02 -61.313391 -61.410329   
394.850000  50    -61.392430  7.777697e-02 -61.314653 -61.422924   
418.625000  51    -61.406694  9.209873e-02 -61.314595 -61.436037   
442.400000  52    -61.421451  1.084365e-01 -61.313015 -61.449492   
466.175000  53    -61.436605  1.268759e-01 -61.309730 -61.463271   
489.950000  54    -61.452060  1.474878e-01 -61.304572 -61.477336   
513.725000  55    -61.467717  1.703378e-01 -61.297379 -61.491626   
537.500000  56    -61.483483  1.954931e-01 -61.287990 -61.505892   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')